In [ ]:
%reload_ext autoreload
%autoreload 2


import functools
print = functools.partial(print, flush=True)

import os
import numpy as np
import pandas as pd
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # for pdfs
import matplotlib.pyplot as plt
from matplotlib import cm
from pathlib import Path
import pickle
from tqdm import tqdm
import scipy

import flexiznam as flz
from cottage_analysis.preprocessing import synchronisation
from cottage_analysis.analysis import spheres, find_depth_neurons, common_utils, fit_gaussian_blob, size_control
from cottage_analysis.plotting import basic_vis_plots, plotting_utils
from cottage_analysis.pipelines import pipeline_utils
from v1_depth_analysis.v1_manuscript_2023 import depth_selectivity, close_loop_rsof

In [ ]:
VERSION = 1
SAVE_ROOT = '/camp/lab/znamenskiyp/home/shared/presentations/v1_manuscript_2023/ver'+str(VERSION)+'/fig3/'
os.makedirs(SAVE_ROOT, exist_ok=True)

In [ ]:
# LOAD DATA
# Load an example session
project='hey2_3d-vision_foodres_20220101'
session_name='PZAH8.2h_S20230116'
flexilims_session = flz.get_flexilims_session(project)

vs_df_example, trials_df_example = spheres.sync_all_recordings(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=project,
    filter_datasets={'anatomical_only':3},
    recording_type="two_photon",
    protocol_base="SpheresPermTubeReward",
    photodiode_protocol=5,
    return_volumes=True,
)

neurons_ds_example = pipeline_utils.create_neurons_ds(
    session_name=session_name, flexilims_session=flexilims_session, project=None, conflicts="skip"
)
neurons_df_example = pd.read_pickle(neurons_ds_example.path_full)

In [ ]:
# Fig. 3A-C: 1 example neuron, binned by RS (A), OF (B), and heatmap (C)
# Fig. 3D-F: another example neuron, binned by RS (D), OF (E), and heatmap (F)
EXAMPLE_ROIS = [
                249, 
                250,
                ]

fig = plt.figure()
for iroi, roi in enumerate(EXAMPLE_ROIS):
    
    depth_selectivity.plot_depth_tuning_curve(
        fig=fig,
        neurons_df=neurons_df_example,
        trials_df=trials_df_example,
        roi=roi,
        rs_thr=0.2,
        plot_fit=False,
        linewidth=3,
        linecolor="k",
        fit_linecolor="r",
        closed_loop=1,
        plot_x=0,
        plot_y=1-0.3*iroi,
        plot_width=0.2,
        plot_height=0.2,
        fontsize_dict={'title':15, 'label':10, 'tick':10},
        )
    if iroi != len(EXAMPLE_ROIS)-1:
        plt.xlabel('')
    
    close_loop_rsof.plot_speed_tuning(
        fig=fig,
        neurons_df=neurons_df_example,
        trials_df=trials_df_example,
        roi=roi,
        is_closed_loop=1,
        nbins=10,
        which_speed="RS",
        speed_min=0.01,
        speed_max=1.5,
        speed_thr=0.01,
        smoothing_sd = 1,
        markersize=4,
        linewidth=2,
        plot_x=0.25,
        plot_y=1-0.3*iroi,
        plot_width=0.2,
        plot_height=0.2,
        fontsize_dict={"title": 15, "label": 10, "tick": 10, "legend": 5},
        legend_on=False,
    )
    if iroi != len(EXAMPLE_ROIS)-1:
        plt.xlabel('')
    plt.ylabel('')
    
    close_loop_rsof.plot_speed_tuning(
        fig=fig,
        neurons_df=neurons_df_example,
        trials_df=trials_df_example,
        roi=roi,
        is_closed_loop=1,
        nbins=10,
        which_speed="OF",
        speed_min=0.01,
        speed_max=1.5,
        speed_thr=0.01,
        smoothing_sd = 1,
        markersize=4,
        linewidth=2,
        plot_x=0.5,
        plot_y=1-0.3*iroi,
        plot_width=0.2,
        plot_height=0.2,
        fontsize_dict={"title": 15, "label": 10, "tick": 10, "legend": 5},
        legend_on=True,
    )
    if iroi != len(EXAMPLE_ROIS)-1:
        plt.xlabel('')
    plt.ylabel('')
    
    if iroi != len(EXAMPLE_ROIS)-1:
        xlabel=''
    else:
        xlabel='Running speed (cm/s)'
    _ = close_loop_rsof.plot_RS_OF_matrix(fig=fig,
                                        trials_df=trials_df_example,
                                        roi=roi,
                                        log_range={
                                            "rs_bin_log_min": 0,
                                            "rs_bin_log_max": 2.5,
                                            "rs_bin_num": 6,
                                            "of_bin_log_min": -1.5,
                                            "of_bin_log_max": 3.5,
                                            "of_bin_num": 11,
                                            "log_base": 10,
                                        },
                                        xlabel=xlabel,
                                        ylabel="Optical flow speed \n(degrees/s)",
                                        plot_x=0.9,
                                        plot_y=1-0.3*iroi,
                                        plot_width=0.2,
                                        plot_height=0.2,
                                        cbar_width=0.01,
                                        fontsize_dict={"title": 15, "label": 10, "tick": 10},
        
    )

# Fig. 3G-H: scatterplot of preferred depth vs. ratio of preferred RS and OF speeds (cross-validated)
# Fig. 3I: scatterplot ratio of preferred RS and OF speeds (cross-validated) colored by preferred depth
# Fig. 3J: scatterplot of preferred depth vs. RS (in different bins of OF speed)

